## Data wrangling - 'master' dataset

In [2]:
import pandas as pd
from pandas import DataFrame

#### Read dataset from file

In [3]:
master: DataFrame = pd.read_csv('./data/master.csv')

#### Split Store_Dept to separate attributes

In [5]:
store_dept = master['Store_Dept'].str.split(pat='_', expand=True)
master['Store'] = store_dept[0]
master['Dept'] = store_dept[1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344667 entries, 0 to 344666
Data columns (total 2 columns):
0    344667 non-null object
1    344667 non-null object
dtypes: object(2)
memory usage: 2.6+ MB


#### Split Month & Year from Date

In [101]:
master['Date'] = pd.to_datetime(master['Date'], format='%m/%d/%y')
master['Year'] = master['Date'].map(lambda x: x.year)
master['Month'] = master['Date'].map(lambda x: x.month)

#### Calculate if the row has anyone of the markdowns populated

In [102]:
no_markdowns = master['MarkDown1'].isna() & master['MarkDown2'].isna() \
        & master['MarkDown3'].isna() & master['MarkDown4'].isna() \
        & master['MarkDown5'].isna()
master['HasMarkDown'] = (no_markdowns == False)

#### Clearing NA values

In [103]:
cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for col in cols:
    master[col] = master[col].fillna(0)

#### Datatype conversion

In [104]:
cols = ['Store', 'Dept', 'Type', 'IsHoliday', 'HasMarkDown', 'Year', 'Month']
for col in cols:
    master[col] = pd.Categorical(master[col])

master = master.drop(columns='Store_Dept')
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344667 entries, 0 to 344666
Data columns (total 19 columns):
Date            344667 non-null datetime64[ns]
Weekly_Sales    344667 non-null float64
IsHoliday       344667 non-null category
Temperature     344667 non-null float64
Fuel_Price      344667 non-null float64
MarkDown1       344667 non-null float64
MarkDown2       344667 non-null float64
MarkDown3       344667 non-null float64
MarkDown4       344667 non-null float64
MarkDown5       344667 non-null float64
CPI             344667 non-null float64
Unemployment    344667 non-null float64
Type            344667 non-null category
Size            344667 non-null int64
Store           344667 non-null category
Dept            344667 non-null category
Year            344667 non-null category
Month           344667 non-null category
HasMarkDown     344667 non-null category
dtypes: category(7), datetime64[ns](1), float64(10), int64(1)
memory usage: 33.9 MB


#### Dummy coding

In [105]:
types = pd.get_dummies(pd.DataFrame({'Type': master['Type']}))
master = pd.concat([master, types], axis=1)

is_holiday = pd.get_dummies(pd.DataFrame({'IsHoliday': master['IsHoliday']}))
master = pd.concat([master, is_holiday], axis=1)

has_markdown = pd.get_dummies(pd.DataFrame({'HasMarkDown': master['HasMarkDown']}))
master = pd.concat([master, has_markdown], axis=1)

month = pd.get_dummies(pd.DataFrame({'Month': master['Month']}))
master = pd.concat([master, month], axis=1)

dept = pd.get_dummies(pd.DataFrame({'Dept': master['Dept']}))
master = pd.concat([master, dept], axis=1)

master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344667 entries, 0 to 344666
Columns: 119 entries, Date to Dept_99
dtypes: category(7), datetime64[ns](1), float64(10), int64(1), uint8(100)
memory usage: 66.7 MB


In [106]:
master.head()

,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Dept_90,Dept_91,Dept_92,Dept_93,Dept_94,Dept_95,Dept_96,Dept_97,Dept_98,Dept_99
0,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
master.to_pickle('./data/master.pickle')
print('DataFrame stored in pickle file...')


DataFrame stored in pickle file...
